In [34]:
economic_states = {
    '通胀上升+增长上升': ['商品', '周期股'],
    '通胀下降+增长上升': ['股票', '信用债'],
    '通胀上升+增长下降': ['黄金', '国债'],
    '通胀下降+增长下降': ['国债', '防御股']
}


import os
import shutil
import pandas as pd
import yfinance as yf
import riskfolio as rp
from pypfopt import risk_models
from pypfopt import EfficientFrontier
from pypfopt import objective_functions

In [35]:
# Ensure directories for backup and operation exist
backup_dir = "backup_data"
operated_dir = "operated_data"
os.makedirs(backup_dir, exist_ok=True)
os.makedirs(operated_dir, exist_ok=True)

# Function to load data from backup or download if not available
def load_or_download_data(filename, tickers, start=None, end=None, period=None):
    backup_path = os.path.join(backup_dir, filename)
    operated_path = os.path.join(operated_dir, filename)

    # Check if backup exists
    if os.path.exists(backup_path):
        print(f"Loading data from backup: {backup_path}")
        # Copy backup to operated directory
        shutil.copy(backup_path, operated_path)
    else:
        print(f"Downloading data for: {tickers}")
        if period:
            data = yf.download(tickers, period=period)['Adj Close']
        else:
            data = yf.download(tickers, start=start, end=end)['Adj Close']
        # Save to backup and operated directories
        data.to_csv(backup_path)
        shutil.copy(backup_path, operated_path)

    # Load data from operated directory
    return pd.read_csv(operated_path, index_col=0, parse_dates=True)

In [36]:

# 获取宏观经济数据（示例：中美30年期国债利差）
data = yf.download(['AAPL'], start='1995-01-01')['Adj Close']

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [37]:
macro_data_file = 'macro_data.csv'
macro_data = load_or_download_data(macro_data_file, ['^FVX', '^TNX'], start='1995-01-01')

Loading data from backup: backup_data\macro_data.csv


C:\Users\hellokitty\AppData\Local\Temp\ipykernel_11204\1376881574.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(operated_path, index_col=0, parse_dates=True)


In [38]:
# 计算美债-中债利差
debt_spread_file = 'debt_spread.csv'
debt_spread = load_or_download_data(debt_spread_file, ['^FVX', '^TNX'], start='1995-01-01')

C:\Users\hellokitty\AppData\Local\Temp\ipykernel_11204\1376881574.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(operated_path, index_col=0, parse_dates=True)


Loading data from backup: backup_data\debt_spread.csv


In [39]:

# 配置四大类资产
assets = ['SPY', 'TLT', 'GLD', 'GSG']
returns_file = 'asset_returns.csv'
returns = load_or_download_data(returns_file, assets, period='30y') # .pct_change().dropna()

Loading data from backup: backup_data\asset_returns.csv


C:\Users\hellokitty\AppData\Local\Temp\ipykernel_11204\1376881574.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.read_csv(operated_path, index_col=0, parse_dates=True)


In [40]:
# Convert all columns in the returns DataFrame to numeric
returns = returns.apply(pd.to_numeric, errors='coerce')

# Calculate percentage change and drop NaN values
returns = returns.pct_change().dropna()

In [41]:
# 风险平价优化
port = rp.Portfolio(returns=returns)
port.assets_stats(method_cov='hist')  # 使用EWMA协方差矩阵
port.rp_optimization(model='Classic', rm='MV', rf=0, b=None, hist=True)  # Corrected arguments

,weights
Close,0.049792
Close.1,0.042262
Close.2,0.072707
Close.3,0.087747
High,0.045573
High.1,0.040279
High.2,0.063015
High.3,0.079275
Low,0.051871
Low.1,0.042031


In [42]:
# assets = ['SPY', 'TLT', 'GLD', 'GSG']
# # 获取多资产数据（含股债商品）
# prices_file = 'prices.csv'
# prices = load_or_download_data(prices_file, assets, start="1995-01-01", end="2025-04-18")
prices = returns

In [33]:
from pypfopt.hierarchical_portfolio import HRPOpt

In [44]:

# 创建HRP优化器
hrp = HRPOpt(returns=returns)

# 改进点：使用Ward方差最小化聚类法
hrp.optimize(linkage_method='ward')
print("分层风险平价权重:", hrp.clean_weights())


分层风险平价权重: OrderedDict({'Close': 0.04807, 'Close.1': 0.02436, 'Close.2': 0.03247, 'Close.3': 0.09045, 'High': 0.07818, 'High.1': 0.03351, 'High.2': 0.07524, 'High.3': 0.05967, 'Low': 0.04911, 'Low.1': 0.02647, 'Low.2': 0.05144, 'Low.3': 0.06203, 'Open': 0.07094, 'Open.1': 0.02284, 'Open.2': 0.07099, 'Open.3': 0.2041, 'Volume': 0.0, 'Volume.1': 0.0, 'Volume.2': 0.0, 'Volume.3': 0.0})


In [ ]:

# plot culster for hrp.clean_weights()
import matplotlib.pyplot as plt
# plt show cluster_map(hrp.cluster_map())



AttributeError: 'HRPOpt' object has no attribute 'plot_dendrogram'

In [52]:

# Handle missing values in the prices DataFrame
prices_cleaned = prices.dropna()


In [54]:

# Ledoit-Wolf 收缩估计（改进协方差矩阵稳定性）
# Ensure there are no missing values in prices_cleaned
prices_cleaned = prices_cleaned.dropna()

# Ledoit-Wolf 收缩估计（改进协方差矩阵稳定性）
# Remove rows where all values are zero
prices_cleaned = prices_cleaned[(prices_cleaned != 0).any(axis=1)]


In [61]:

# 结合半定规划优化
hrp = HRPOpt(returns=prices_cleaned) #.pct_change())
hrp.optimize()

# 创建风险平价与均值方差混合模型
# Calculate the covariance matrix from the returns DataFrame
cov_matrix = prices_cleaned.cov()

ef = EfficientFrontier(None, cov_matrix)  # 不依赖收益率预测
# 添加无风险利率目标函数  
# ef.add_objective(objective_functions.risk_free_rate, risk_free_rate=0.02)  # 设置无风险利率
# ef.add_objective(objective_functions.risk_aversion, risk_aversion=0.95)  # 设置风险厌恶系数 

In [63]:



# 添加流动性约束（假设流动性数据）
liquidity = {"SPY": 0.9, "TLT": 0.7, "GLD": 0.6, "GSG": 0.5}
ef.add_constraint(lambda w: sum([w[i] * liquidity[asset] for i, asset in enumerate(assets)]) >= 0.8)


In [72]:

# 求解优化问题
weights = ef.nonconvex_objective(lambda w: objective_functions.sharpe_ratio(w, returns, cov_matrix), weights_sum_to_one=True)


TypeError: <lambda>() takes 1 positional argument but 2 were given

In [73]:

# 定义随时间变化的风险预算
def dynamic_risk_budget(returns):
    # 基于波动率regime调整预算
    recent_vol = returns.iloc[-60:].std()
    return recent_vol / recent_vol.sum()

# 应用时变风险预算
returns_cleaned = returns.dropna()
hrp = HRPOpt(returns_cleaned)
hrp.optimize(risk_budget=dynamic_risk_budget(hrp.returns))

# 分层风险平价回测
perf = hrp.portfolio_performance(verbose=True)

# 与传统方法对比
rp = rp.RiskParityPortfolio(cov_matrix=cov_matrix)
rp.optimize()
print("传统风险平价:", rp.weights)

# 绘制有效前沿对比
ef = EfficientFrontier(None, cov_matrix)
ef.efficient_risk(target_risk=0.15)
ef.portfolio_performance(verbose=True)

TypeError: HRPOpt.optimize() got an unexpected keyword argument 'risk_budget'